In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import re
from spacytextblob.spacytextblob import SpacyTextBlob
from tqdm import tqdm
from tqdm.auto import tqdm 

plt.rcParams["figure.figsize"] = (12,8)
tqdm.pandas()

In [10]:
tweet_data = pd.read_csv('Data/tweet_data.csv')
tweet_data

,Unnamed: 0,User ID,Tweet ID,Text,Date Only,Date,lat,lon,Sentiment,Subjectivity,...,Chile Mentioned,Pray Mentioned,Donate Mentioned,Obama Mentioned,US Mentioned,Thoughts and Prayers,Is Retweet,Is Reply,Directed ats,Hashtags Used
0,0,22077441,9807469480,Planet earth is truely amazing!!! I love watch...,2010-02-28,2010-02-28 21:25:41,43.009815,-83.710408,0.854167,0.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,22077441,9805735171,Aww I want someone to say this to me one day :...,2010-02-28,2010-02-28 20:44:41,43.009815,-83.710408,0.211111,0.688889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,22077441,9800834089,I love these cheetoes commercials!! Too funny...,2010-02-28,2010-02-28 18:43:00,43.009815,-83.710408,0.515625,0.800000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,22077441,9797095168,Aww my lil sis got her first pattern so she ca...,2010-02-28,2010-02-28 17:09:03,43.009815,-83.710408,0.262500,0.583333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,22077441,9796762943,I HATE being sick.. It sucks all of the energy...,2010-02-28,2010-02-28 17:02:09,43.009815,-83.710408,-0.604762,0.685714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450509,2450509,73016661,8082730603,@NYCsocialit3 nun u,2010-01-22,2010-01-22 14:14:30,40.627262,-73.926384,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2450510,2450510,73016661,8081316956,RT @BonesThisWorld: A trip too harlem for a fe...,2010-01-22,2010-01-22 13:28:51,40.627262,-73.926384,-0.200000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2450511,2450511,73016661,8076088351,@Uniquehope was good,2010-01-22,2010-01-22 10:53:13,40.627262,-73.926384,0.700000,0.600000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2450512,2450512,73016661,8075627082,@DaStar6MFM lmaooo yea dat shit is crazy,2010-01-22,2010-01-22 10:40:05,40.627262,-73.926384,-0.400000,0.850000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [11]:
data_to_agg = tweet_data.drop(columns=['Tweet ID', 'User ID', 'Text', 'lat','lon'])
data_to_agg

,Unnamed: 0,Date Only,Date,Sentiment,Subjectivity,Haiti Mentioned,Chile Mentioned,Pray Mentioned,Donate Mentioned,Obama Mentioned,US Mentioned,Thoughts and Prayers,Is Retweet,Is Reply,Directed ats,Hashtags Used
0,0,2010-02-28,2010-02-28 21:25:41,0.854167,0.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2010-02-28,2010-02-28 20:44:41,0.211111,0.688889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,2010-02-28,2010-02-28 18:43:00,0.515625,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2010-02-28,2010-02-28 17:09:03,0.262500,0.583333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2010-02-28,2010-02-28 17:02:09,-0.604762,0.685714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450509,2450509,2010-01-22,2010-01-22 14:14:30,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2450510,2450510,2010-01-22,2010-01-22 13:28:51,-0.200000,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2450511,2450511,2010-01-22,2010-01-22 10:53:13,0.700000,0.600000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2450512,2450512,2010-01-22,2010-01-22 10:40:05,-0.400000,0.850000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [12]:
date_tweet_count = data_to_agg.groupby('Date Only').size()
date_tweet_count.name = "Tweet Count"
date_tweet_count

Date Only
2010-01-01    16775
2010-01-02    20002
2010-01-03    21719
2010-01-04    23945
2010-01-05    25530
2010-01-06    26208
2010-01-07    28287
2010-01-08    27316
2010-01-09    25206
2010-01-10    24248
2010-01-11    26946
2010-01-12    27179
2010-01-13    27749
2010-01-14    28889
2010-01-15    28136
2010-01-16    27446
2010-01-17    28953
2010-01-18    27833
2010-01-19    31703
2010-01-20    31405
2010-01-21    32044
2010-01-22    33808
2010-01-23    28406
2010-01-24    37627
2010-01-25    33576
2010-01-26    35721
2010-01-27    36666
2010-01-28    38207
2010-01-29    38255
2010-01-30    36266
2010-01-31    51127
2010-02-01    38656
2010-02-02    40710
2010-02-03    40922
2010-02-04    42724
2010-02-05    45404
2010-02-06    41459
2010-02-07    53394
2010-02-08    42762
2010-02-09    49089
2010-02-10    57256
2010-02-11    52118
2010-02-12    53395
2010-02-13    46787
2010-02-14    46624
2010-02-15    46190
2010-02-16    56668
2010-02-17    58477
2010-02-18    65562
2010-02-19

In [14]:
date_mentioned_counts = data_to_agg.drop(columns=['Subjectivity', 'Sentiment']).groupby('Date Only').sum()
date_mentioned_counts

,Unnamed: 0,Date,Haiti Mentioned,Chile Mentioned,Pray Mentioned,Donate Mentioned,Obama Mentioned,US Mentioned,Thoughts and Prayers,Is Retweet,Is Reply,Directed ats,Hashtags Used
Date Only,,,,,,,,,,,,,
2010-01-01,20364055438,2010-01-01 10:45:212010-01-01 09:14:402010-01-...,14.0,1.0,43.0,2.0,18.0,143.0,3.0,1860.0,5995.0,12503.0,1735.0
2010-01-02,23883073007,2010-01-02 00:00:002010-01-02 00:00:002010-01-...,1.0,4.0,66.0,4.0,12.0,153.0,0.0,2336.0,7289.0,13851.0,2779.0
2010-01-03,26182014905,2010-01-03 00:00:002010-01-03 00:00:002010-01-...,2.0,4.0,86.0,11.0,21.0,235.0,1.0,2445.0,8162.0,14906.0,2961.0
2010-01-04,29434187763,2010-01-04 00:00:002010-01-04 00:00:002010-01-...,8.0,3.0,88.0,4.0,28.0,297.0,4.0,2817.0,9309.0,17305.0,3019.0
2010-01-05,31208758631,2010-01-05 00:00:002010-01-05 21:57:032010-01-...,16.0,5.0,82.0,8.0,26.0,353.0,0.0,3106.0,10067.0,18771.0,3466.0
2010-01-06,32002430008,2010-01-06 00:00:002010-01-06 00:00:002010-01-...,4.0,4.0,75.0,22.0,32.0,249.0,1.0,3226.0,10095.0,19180.0,3455.0
2010-01-07,34388190146,2010-01-07 00:00:002010-01-07 00:00:002010-01-...,10.0,3.0,105.0,10.0,46.0,290.0,0.0,3352.0,10938.0,20323.0,3124.0
2010-01-08,33179264640,2010-01-08 23:56:422010-01-08 23:55:242010-01-...,9.0,7.0,55.0,10.0,28.0,380.0,0.0,3699.0,10001.0,26950.0,4584.0
2010-01-09,30790151886,2010-01-09 21:12:382010-01-09 20:55:022010-01-...,6.0,2.0,51.0,8.0,25.0,254.0,0.0,3236.0,9073.0,18157.0,2868.0


In [15]:
date_sentiment_avg = data_to_agg.groupby('Date Only')[['Sentiment','Subjectivity']].mean()
date_sentiment_avg

,Sentiment,Subjectivity
Date Only,,
2010-01-01,0.164221,0.407985
2010-01-02,0.133074,0.392497
2010-01-03,0.126781,0.394785
2010-01-04,0.129575,0.394949
2010-01-05,0.123082,0.395139
2010-01-06,0.131754,0.392583
2010-01-07,0.124644,0.384818
2010-01-08,0.133814,0.378189
2010-01-09,0.127111,0.384170


In [23]:
all_date_data = date_sentiment_avg.merge(date_mentioned_counts, left_on='Date Only', right_on='Date Only')
all_date_data = all_date_data.merge(date_tweet_count, left_on='Date Only', right_on='Date Only')
all_date_data

,Sentiment,Subjectivity,Unnamed: 0,Date,Haiti Mentioned,Chile Mentioned,Pray Mentioned,Donate Mentioned,Obama Mentioned,US Mentioned,Thoughts and Prayers,Is Retweet,Is Reply,Directed ats,Hashtags Used,Tweet Count
Date Only,,,,,,,,,,,,,,,,
2010-01-01,0.164221,0.407985,20364055438,2010-01-01 10:45:212010-01-01 09:14:402010-01-...,14.0,1.0,43.0,2.0,18.0,143.0,3.0,1860.0,5995.0,12503.0,1735.0,16775
2010-01-02,0.133074,0.392497,23883073007,2010-01-02 00:00:002010-01-02 00:00:002010-01-...,1.0,4.0,66.0,4.0,12.0,153.0,0.0,2336.0,7289.0,13851.0,2779.0,20002
2010-01-03,0.126781,0.394785,26182014905,2010-01-03 00:00:002010-01-03 00:00:002010-01-...,2.0,4.0,86.0,11.0,21.0,235.0,1.0,2445.0,8162.0,14906.0,2961.0,21719
2010-01-04,0.129575,0.394949,29434187763,2010-01-04 00:00:002010-01-04 00:00:002010-01-...,8.0,3.0,88.0,4.0,28.0,297.0,4.0,2817.0,9309.0,17305.0,3019.0,23945
2010-01-05,0.123082,0.395139,31208758631,2010-01-05 00:00:002010-01-05 21:57:032010-01-...,16.0,5.0,82.0,8.0,26.0,353.0,0.0,3106.0,10067.0,18771.0,3466.0,25530
2010-01-06,0.131754,0.392583,32002430008,2010-01-06 00:00:002010-01-06 00:00:002010-01-...,4.0,4.0,75.0,22.0,32.0,249.0,1.0,3226.0,10095.0,19180.0,3455.0,26208
2010-01-07,0.124644,0.384818,34388190146,2010-01-07 00:00:002010-01-07 00:00:002010-01-...,10.0,3.0,105.0,10.0,46.0,290.0,0.0,3352.0,10938.0,20323.0,3124.0,28287
2010-01-08,0.133814,0.378189,33179264640,2010-01-08 23:56:422010-01-08 23:55:242010-01-...,9.0,7.0,55.0,10.0,28.0,380.0,0.0,3699.0,10001.0,26950.0,4584.0,27316
2010-01-09,0.127111,0.384170,30790151886,2010-01-09 21:12:382010-01-09 20:55:022010-01-...,6.0,2.0,51.0,8.0,25.0,254.0,0.0,3236.0,9073.0,18157.0,2868.0,25206


In [24]:
all_date_data = all_date_data.drop(columns = ["Date","Unnamed: 0"])
all_date_data

,Sentiment,Subjectivity,Haiti Mentioned,Chile Mentioned,Pray Mentioned,Donate Mentioned,Obama Mentioned,US Mentioned,Thoughts and Prayers,Is Retweet,Is Reply,Directed ats,Hashtags Used,Tweet Count
Date Only,,,,,,,,,,,,,,
2010-01-01,0.164221,0.407985,14.0,1.0,43.0,2.0,18.0,143.0,3.0,1860.0,5995.0,12503.0,1735.0,16775
2010-01-02,0.133074,0.392497,1.0,4.0,66.0,4.0,12.0,153.0,0.0,2336.0,7289.0,13851.0,2779.0,20002
2010-01-03,0.126781,0.394785,2.0,4.0,86.0,11.0,21.0,235.0,1.0,2445.0,8162.0,14906.0,2961.0,21719
2010-01-04,0.129575,0.394949,8.0,3.0,88.0,4.0,28.0,297.0,4.0,2817.0,9309.0,17305.0,3019.0,23945
2010-01-05,0.123082,0.395139,16.0,5.0,82.0,8.0,26.0,353.0,0.0,3106.0,10067.0,18771.0,3466.0,25530
2010-01-06,0.131754,0.392583,4.0,4.0,75.0,22.0,32.0,249.0,1.0,3226.0,10095.0,19180.0,3455.0,26208
2010-01-07,0.124644,0.384818,10.0,3.0,105.0,10.0,46.0,290.0,0.0,3352.0,10938.0,20323.0,3124.0,28287
2010-01-08,0.133814,0.378189,9.0,7.0,55.0,10.0,28.0,380.0,0.0,3699.0,10001.0,26950.0,4584.0,27316
2010-01-09,0.127111,0.384170,6.0,2.0,51.0,8.0,25.0,254.0,0.0,3236.0,9073.0,18157.0,2868.0,25206


In [26]:
all_date_data.to_csv("Data/date_data.csv")